# 1st Attempt
Optimize the model by chaning the bins and inputs in the data without changing the model first, to see if there is any value in changing what was passed into the model.

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' and also drop the STATUS column.
application_df = application_df.drop(['EIN', 'NAME', 'STATUS'], axis=1)
application_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0   Association              0                      N     5000              1  
1  Co-operative         1-9999                      N   108590              1  
2   Association              0                      N     5000              0  
3         Trust    10000-24999                      N     6692              1  
4         Trust  100000-499999                      N   142590              1

In [4]:
# Choose original cutoff value for 'APPLICATION_TYPE' and create a list of application types to be replaced
application_types_to_replace = ["T9", "T13", "T12", "T2", "T14", "T25", "T15", "T29", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose new cutoff value for CLASSIFICATION that includes one more bin
classifications_to_replace = []
for x in application_df["CLASSIFICATION"]:
    if (x != "C1000") and (x != "C2000") and (x != "C1200") and (x != "C3000") and (x != "C2100") and (x != "C7000"):
        if x not in classifications_to_replace:
            classifications_to_replace.append(x)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [6]:
# Look at INCOME_AMT value counts for possible binning of outliers
application_df["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [7]:
# Look at AFFILIATION value counts for possible binning of outliers
application_df["AFFILIATION"].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [8]:
# Choose cutoff value for 'AFFILIATION' and create a list of affiliation types to be replaced
affiliation_types_to_replace = ["National", "Regional", "Other"]

# Replace in dataframe
for aff in affiliation_types_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
Other                  50
Name: AFFILIATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, prefix=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
application_df.head()

ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0     5000              1                       0                     1   
1   108590              1                       0                     0   
2     5000              0                       0                     0   
3     6692              1                       0                     0   
4   142590              1                       0                     0   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                     0                    0                    0   
1                     0                    1                    0   
2                     0                    0                    0   
3                     0                    1                    0   
4                     0                    1                    0   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  ...   
2                    1                    0                    0  ...   
3                    0                    0                    0  ...   
4                    0                    0                    0  ...   

   INCOME_AMT_1-9999  INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  \
0                  0                       0                         0   
1                  1                       0                         0   
2                  0                       0                         0   
3                  0                       1                         0   
4                  0                       0                         1   

   INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  \
0                   0                 0                       0   
1                   0                 0                       0   
2                   0                 0                       0   
3                   0                 0                       0   
4                   0                 0                       0   

   INCOME_AMT_50M+  INCOME_AMT_5M-10M  SPECIAL_CONSIDERATIONS_N  \
0                0                  0                         1   
1                0                  0                         1   
2                0                  0                         1   
3                0                  0                         1   
4                0                  0                         1   

   SPECIAL_CONSIDERATIONS_Y  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  

[5 rows x 42 columns]

In [10]:
# Split our preprocessed data into our features and target arrays

# Target
y = application_df["IS_SUCCESSFUL"]

# Features
X = application_df.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Compile, Train, and Evaluate a model that is identical to the original, but with the changes that have been made to the dataset
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

# Train the model
cp = tf.keras.callbacks.ModelCheckpoint('Output/weights_mod1.h5', save_weights_only=True, period=5)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3360      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,821
Trainable params: 5,821
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 1s 583us/step - loss: 0.5705 - accuracy: 0.7190
Epoch 2/100
804/804 [==============================] - 0s 584us/step - loss: 0.5549 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 0s 578us/step - loss: 0.5518 - accuracy: 0.7295
Epoch 4/1

804/804 [==============================] - 0s 581us/step - loss: 0.5356 - accuracy: 0.7388
Epoch 71/100
804/804 [==============================] - 0s 576us/step - loss: 0.5351 - accuracy: 0.7384
Epoch 72/100
804/804 [==============================] - 0s 571us/step - loss: 0.5351 - accuracy: 0.7392
Epoch 73/100
804/804 [==============================] - 0s 577us/step - loss: 0.5353 - accuracy: 0.7403
Epoch 74/100
804/804 [==============================] - 0s 569us/step - loss: 0.5350 - accuracy: 0.7395
Epoch 75/100
804/804 [==============================] - 0s 579us/step - loss: 0.5346 - accuracy: 0.7393
Epoch 76/100
804/804 [==============================] - 0s 577us/step - loss: 0.5353 - accuracy: 0.7388
Epoch 77/100
804/804 [==============================] - 0s 573us/step - loss: 0.5351 - accuracy: 0.7400
Epoch 78/100
804/804 [==============================] - 0s 588us/step - loss: 0.5346 - accuracy: 0.7406
Epoch 79/100
804/804 [==============================] - 0s 579us/step - loss:

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5573 - accuracy: 0.7298 - 197ms/epoch - 737us/step
Loss: 0.5572628378868103, Accuracy: 0.7297959327697754


In [14]:
# Export our model to HDF5 file
nn.save("Output/AlphabetSoupCharity_Optimization_mod1.h5")

## Results
The Accuracy is practically the same as before, but slightly higher. The original was 0.7280466556549072 and this new model is 0.7297959327697754. The slight increase is enough reason to keep these changes as we move forward.

# 2nd Attempt
Using the same processed data from the first attempt, we will now alter the number of hidden layers and nodes to see how the model reacts.

In [93]:
# Compile, Train, and Evaluate a model using the same data as the prior model, but with changes to number of hidden layers and nodes
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

# Train the model
cp = tf.keras.callbacks.ModelCheckpoint('Output/weights_mod2.h5', save_weights_only=True, period=5)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp])

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_149 (Dense)           (None, 256)               10752     
                                                                 
 dense_150 (Dense)           (None, 128)               32896     
                                                                 
 dense_151 (Dense)           (None, 64)                8256      
                                                                 
 dense_152 (Dense)           (None, 1)                 65        
                                                                 
Total params: 51,969
Trainable params: 51,969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 1s 773us/step - loss: 0.5683 - accuracy: 0.7220
Epoch 2/100
804/804 [==============================] - 1s 811us/step - loss: 0

804/804 [==============================] - 1s 748us/step - loss: 0.5319 - accuracy: 0.7407
Epoch 70/100
804/804 [==============================] - 1s 754us/step - loss: 0.5333 - accuracy: 0.7405
Epoch 71/100
804/804 [==============================] - 1s 752us/step - loss: 0.5331 - accuracy: 0.7410
Epoch 72/100
804/804 [==============================] - 1s 752us/step - loss: 0.5321 - accuracy: 0.7404
Epoch 73/100
804/804 [==============================] - 1s 757us/step - loss: 0.5315 - accuracy: 0.7411
Epoch 74/100
804/804 [==============================] - 1s 749us/step - loss: 0.5319 - accuracy: 0.7407
Epoch 75/100
804/804 [==============================] - 1s 760us/step - loss: 0.5317 - accuracy: 0.7408
Epoch 76/100
804/804 [==============================] - 1s 748us/step - loss: 0.5316 - accuracy: 0.7401
Epoch 77/100
804/804 [==============================] - 1s 751us/step - loss: 0.5328 - accuracy: 0.7406
Epoch 78/100
804/804 [==============================] - 1s 747us/step - loss:

In [94]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5950 - accuracy: 0.7312 - 181ms/epoch - 676us/step
Loss: 0.5949540138244629, Accuracy: 0.731195330619812


In [95]:
# Export our model to HDF5 file
nn.save("Output/AlphabetSoupCharity_Optimization_mod2.h5")

## Results
The Accuracy increased slightly by adding another hidden layer and increasing the number of nodes. The original accuracy was 0.7297959327697754 and the new accuracy is 0.731195330619812. Multiple attempts were made at chaning the number of nodes and layers, but what was chosen is the result of finding the best of those options.

# 3rd Attempt
The number of epochs will be changed from the previous model to see if a difference is made. Type of activation was tested previously (though not shown in the work here) and did not improve the model so epochs appears to be the last thing to test.

In [106]:
# Compile, Train, and Evaluate a model using the same data as the prior model, but with changes to number of epochs
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

# Train the model
cp = tf.keras.callbacks.ModelCheckpoint('Output/weights_mod3.h5', save_weights_only=True, period=5)
fit_model = nn.fit(X_train_scaled,y_train,epochs=500,callbacks=[cp])

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_173 (Dense)           (None, 256)               10752     
                                                                 
 dense_174 (Dense)           (None, 128)               32896     
                                                                 
 dense_175 (Dense)           (None, 64)                8256      
                                                                 
 dense_176 (Dense)           (None, 1)                 65        
                                                                 
Total params: 51,969
Trainable params: 51,969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
804/804 [==============================] - 1s 765us/step - loss: 0.5673 - accuracy: 0.7223
Epoch 2/500
804/804 [==============================] - 1s 768us/step - loss: 0

804/804 [==============================] - 1s 749us/step - loss: 0.5319 - accuracy: 0.7396
Epoch 70/500
804/804 [==============================] - 1s 752us/step - loss: 0.5308 - accuracy: 0.7399
Epoch 71/500
804/804 [==============================] - 1s 782us/step - loss: 0.5311 - accuracy: 0.7404
Epoch 72/500
804/804 [==============================] - 1s 766us/step - loss: 0.5315 - accuracy: 0.7407
Epoch 73/500
804/804 [==============================] - 1s 776us/step - loss: 0.5308 - accuracy: 0.7405
Epoch 74/500
804/804 [==============================] - 1s 747us/step - loss: 0.5313 - accuracy: 0.7399
Epoch 75/500
804/804 [==============================] - 1s 753us/step - loss: 0.5317 - accuracy: 0.7407
Epoch 76/500
804/804 [==============================] - 1s 757us/step - loss: 0.5314 - accuracy: 0.7400
Epoch 77/500
804/804 [==============================] - 1s 785us/step - loss: 0.5317 - accuracy: 0.7410
Epoch 78/500
804/804 [==============================] - 1s 798us/step - loss:

804/804 [==============================] - 1s 774us/step - loss: 0.5294 - accuracy: 0.7420
Epoch 148/500
804/804 [==============================] - 1s 874us/step - loss: 0.5298 - accuracy: 0.7419
Epoch 149/500
804/804 [==============================] - 1s 806us/step - loss: 0.5301 - accuracy: 0.7418
Epoch 150/500
804/804 [==============================] - 1s 764us/step - loss: 0.5293 - accuracy: 0.7416
Epoch 151/500
804/804 [==============================] - 1s 766us/step - loss: 0.5295 - accuracy: 0.7415
Epoch 152/500
804/804 [==============================] - 1s 780us/step - loss: 0.5296 - accuracy: 0.7402
Epoch 153/500
804/804 [==============================] - 1s 769us/step - loss: 0.5293 - accuracy: 0.7418
Epoch 154/500
804/804 [==============================] - 1s 763us/step - loss: 0.6207 - accuracy: 0.7418
Epoch 155/500
804/804 [==============================] - 1s 776us/step - loss: 0.5430 - accuracy: 0.7407
Epoch 156/500
804/804 [==============================] - 1s 776us/ste

804/804 [==============================] - 1s 798us/step - loss: 0.5297 - accuracy: 0.7424
Epoch 226/500
804/804 [==============================] - 1s 766us/step - loss: 0.5293 - accuracy: 0.7417
Epoch 227/500
804/804 [==============================] - 1s 756us/step - loss: 0.5283 - accuracy: 0.7430
Epoch 228/500
804/804 [==============================] - 1s 898us/step - loss: 0.5288 - accuracy: 0.7416
Epoch 229/500
804/804 [==============================] - 1s 829us/step - loss: 0.5291 - accuracy: 0.7423
Epoch 230/500
804/804 [==============================] - 1s 886us/step - loss: 0.5282 - accuracy: 0.7420
Epoch 231/500
804/804 [==============================] - 1s 782us/step - loss: 0.5283 - accuracy: 0.7418
Epoch 232/500
804/804 [==============================] - 1s 787us/step - loss: 0.5298 - accuracy: 0.7420
Epoch 233/500
804/804 [==============================] - 1s 768us/step - loss: 0.5365 - accuracy: 0.7417
Epoch 234/500
804/804 [==============================] - 1s 782us/ste

804/804 [==============================] - 1s 763us/step - loss: 0.5281 - accuracy: 0.7424
Epoch 304/500
804/804 [==============================] - 1s 760us/step - loss: 0.5277 - accuracy: 0.7427
Epoch 305/500
804/804 [==============================] - 1s 764us/step - loss: 0.5279 - accuracy: 0.7421
Epoch 306/500
804/804 [==============================] - 1s 760us/step - loss: 0.5287 - accuracy: 0.7423
Epoch 307/500
804/804 [==============================] - 1s 763us/step - loss: 0.5278 - accuracy: 0.7426
Epoch 308/500
804/804 [==============================] - 1s 753us/step - loss: 0.5296 - accuracy: 0.7427
Epoch 309/500
804/804 [==============================] - 1s 755us/step - loss: 0.5282 - accuracy: 0.7426
Epoch 310/500
804/804 [==============================] - 1s 769us/step - loss: 0.5317 - accuracy: 0.7423
Epoch 311/500
804/804 [==============================] - 1s 759us/step - loss: 0.5281 - accuracy: 0.7428
Epoch 312/500
804/804 [==============================] - 1s 755us/ste

804/804 [==============================] - 1s 770us/step - loss: 0.5276 - accuracy: 0.7420
Epoch 382/500
804/804 [==============================] - 1s 891us/step - loss: 0.5277 - accuracy: 0.7423
Epoch 383/500
804/804 [==============================] - 1s 772us/step - loss: 0.5279 - accuracy: 0.7418
Epoch 384/500
804/804 [==============================] - 1s 769us/step - loss: 0.5287 - accuracy: 0.7427
Epoch 385/500
804/804 [==============================] - 1s 764us/step - loss: 0.5277 - accuracy: 0.7420
Epoch 386/500
804/804 [==============================] - 1s 757us/step - loss: 0.5274 - accuracy: 0.7421
Epoch 387/500
804/804 [==============================] - 1s 752us/step - loss: 0.5274 - accuracy: 0.7425
Epoch 388/500
804/804 [==============================] - 1s 752us/step - loss: 0.5284 - accuracy: 0.7423
Epoch 389/500
804/804 [==============================] - 1s 753us/step - loss: 0.5272 - accuracy: 0.7423
Epoch 390/500
804/804 [==============================] - 1s 756us/ste

804/804 [==============================] - 1s 765us/step - loss: 0.5267 - accuracy: 0.7426
Epoch 460/500
804/804 [==============================] - 1s 781us/step - loss: 0.5267 - accuracy: 0.7427
Epoch 461/500
804/804 [==============================] - 1s 768us/step - loss: 0.5269 - accuracy: 0.7429
Epoch 462/500
804/804 [==============================] - 1s 764us/step - loss: 0.5305 - accuracy: 0.7422
Epoch 463/500
804/804 [==============================] - 1s 764us/step - loss: 0.5270 - accuracy: 0.7430
Epoch 464/500
804/804 [==============================] - 1s 762us/step - loss: 0.5268 - accuracy: 0.7429
Epoch 465/500
804/804 [==============================] - 1s 778us/step - loss: 0.5266 - accuracy: 0.7425
Epoch 466/500
804/804 [==============================] - 1s 832us/step - loss: 0.5274 - accuracy: 0.7432
Epoch 467/500
804/804 [==============================] - 1s 788us/step - loss: 0.5266 - accuracy: 0.7428
Epoch 468/500
804/804 [==============================] - 1s 783us/ste

In [107]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7458 - accuracy: 0.7289 - 183ms/epoch - 681us/step
Loss: 0.7458173632621765, Accuracy: 0.728863000869751


In [108]:
# Export our model to HDF5 file
nn.save("Output/AlphabetSoupCharity_Optimization_mod3.h5")

## Results
Increasing the number of Epochs only decreased the accuracy of the model. It was also tested with a smaller number of Epochs with similar results.

# Testing other models to solve this classification problem

In [109]:
# Logistic Regression model
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train_scaled, y_train)
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.7220494479863163
Testing Data Score: 0.7203498542274053


In [112]:
# Random Forest Classifier
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

clfs = []
scores = []
for i in range(50):
    X_train_bootstrap, y_train_bootstrap = resample(X_train_scaled, y_train, random_state=i)
    clf = DecisionTreeClassifier(splitter='random', random_state=i+200).fit(X_train_bootstrap, y_train_bootstrap)
    clfs.append(clf)
    y_preds = [clf.predict(X_test) for clf in clfs]
    y_pred = pd.DataFrame(y_preds).median().round()
    score = score = accuracy_score(y_test, y_pred)
    scores.append(score)
clf = RandomForestClassifier(random_state=1, n_estimators=50).fit(X_train_scaled, y_train)
print(f"Training Data Score: {clf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {clf.score(X_test_scaled, y_test)}")

Training Data Score: 0.8154641579847614
Testing Data Score: 0.7154518950437317


## Results
Neither Logistic Regression nor Random Forest performed better than Mod2 of the Neural Network, but Logistic Regression came close with far less processing needed.